In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from neuralnetwork.myvggnet import myVGGNet
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use("Agg")
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os


Using TensorFlow backend.


In [ ]:
# construct the argument parse and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", required=True,
    help="path to input dataset (i.e., directory of images)")
ap.add_argument("-m", "--model", required=True,
    help="path to output model")
ap.add_argument("-l", "--labelbin", required=True,
    help="path to output label binarizer")
ap.add_argument("-p", "--plot", type=str, default="plot.png",
    help="path to output accuracy/loss plot")
args = vars(ap.parse_args())

In [ ]:
epochs=100
itit_lr=0.001
bs=32
image_dims=(96,96,3)

#initialize the data and labels
data=[]
labels=[]

#grab the image paths and randomly shuffle them

print("Loading images...")
imagePath=""
random.seed(42)
random.shuffle(imagePaths)



In [ ]:
#Loop over the input images
for imagePath in imagePaths:
    image=cv2.imread(imagePath)
    image=cv2.resize(image,(image_dims[1],image_dims[1]))
    img=img_to_array(images)
    data.append(image)
    
    #extract class label from image path and update labels path
    label=imagePath.split(os.path.sep)[-2]
    labels.append(label)
    


In [2]:
#scale the raw pixel intensities
data=np.array(data,dtype="float")/255.0
labels=np.array(labels)
print("[INFO] data matrix: {:.2f}MB".format(
    data.nbytes / (1024 * 1000.0)))

#binarize the models
lb=LabelBinarizer()
labels=lb.fit_transform(labels)

#partition the data into train and test
(trainX,testX,trainY,testY)=train_test_split(data,labels,test_size=0.2,random_state=42)

'#scale the raw pixel intensities\ndata=np.array(data,dtype="float")/255.0\nlabels=np.array(labels)\nprint("[INFO] data matrix: {:.2f}MB".format(\n    data.nbytes / (1024 * 1000.0)))\n\n#binarize the models\nlb=LabelBinarizer()\nlabels=lb.fit_transform(labels)\n\n#partition the data into train and test\n(trainX,testX,trainY,testY)=train_test_split(data,labels,test_size=0.2,random_state=42)'

In [9]:
#Construct image generator for data augmentation
aug=ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")

In [ ]:
#here we create an instance of cnn and choose Adam as our optimization method and Categorical Cross Entropy as loss parameter.
#Had this been a v
print("[INFO] compiling model...")
model = myVGGNet.build(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0],
    depth=IMAGE_DIMS[2], classes=len(lb.classes_))
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
    metrics=["accuracy"])
 
# train the network
print("[INFO] training network...")
H = model.fit_generator(
    aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS, verbose=1)

In [ ]:
print("[info] serializing network...")
model.save(args["model"])
#save the label binarizer..
f=open(args["labelbin"],"wb")
f.write(pickle.dumps(lb))
f.close()

In [ ]:
#Plot Training loss and accuracy..
plt.style.use("ggplot")
plt.figure()
N=epochs
plt.plot(np.arange(0,N),H.history["loss"],label="train_loss")
plt.plot(np.arange(0,N),H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0,N),H.history["acc"], label="train_acc")
plt.plot(np.arange(0,N),H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
plt.show()